In [ ]:
import sfacts as sf

In [ ]:
%%bash
sfacts simulate \
    --model-structure=ssdd3_with_error \
    --num-strains=10 --num-samples=50 --num-positions=5000 \
    --hyperparameters
        gamma_hyper=1e-5 rho_hyper=10.0 pi_hyper=0.1 \
        mu_hyper_mean=10.0 mu_hyper_scale=3.0 m_hyper_concentration=3.0 \
        epsilon_hyper_mode=0.01 epsilon_hyper_spread=1.5 \
        alpha_hyper_mean=100 alpha_hyper_scale=0.5 \
    --random-seed=0 \
    --outpath sim.world.nc

In [ ]:
sim = sf.data.World.load('sim.world.nc')
sim_head = sim.isel(position=slice(0, 500))
sf.plot.plot_community(sim_head, row_linkage_func=lambda w: w.genotypes.linkage("strain"), col_linkage_func=lambda w: w.metagenotypes.linkage("sample"))
sf.plot.plot_genotype(sim_head, row_linkage_func=lambda w: w.genotypes.linkage("strain"))
sf.plot.plot_metagenotype(sim_head, col_linkage_func=lambda w: w.metagenotypes.linkage("sample"))

In [ ]:
%%bash
sfacts dump --tsv sim.world.nc --metagenotype sim.mgen.tsv

head sim.mgen.tsv | column -t

In [ ]:
%%bash

sfacts load_mgen sim.mgen.tsv sim.mgen.nc